In [ ]:

import pickle
from speclearn.tools.data_tools import *
from speclearn.plot.map import *
from speclearn.io.data.aoi import get_full_map_aoi
from speclearn.deep_learning.predict import get_data, predict_full_map
from speclearn.deep_learning.model_utils import load_beta_VAE_model, get_colorbar
from speclearn.deep_learning.ml_tools import (get_poles_aoi)
from sklearn.cluster import KMeans
from matplotlib.text import Text
from speclearn.deep_learning.cluster import cluster_with_kmeans
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from speclearn.io.transform.bin_tools import make_bins
import matplotlib
local_wavelength = select_wavelength(s_0=0, s_1=-12)
import datetime
print('Current time: ', datetime.datetime.now())
figure_dir = '/home/freya/Documents/figures'
from speclearn.tools.constants import *

In [2]:
k = 5
crs = False
area = 'large'
norm = False

In [ ]:
cmap = get_colorbar(k)
aois = get_full_map_aoi(d_long=0.05, d_lat=0.05, step_size=10)
model, model_name = load_beta_VAE_model(crs=crs, norm=norm)
print('model:', model_name)
predict_full_map(aois, model, model_name, crs=crs, norm=norm)
data_2d, coord, latent = get_data(aois[2:-2], model_name, crs=crs, norm=norm) #-2:2
if area == 'small':
    data_2d_s, coord_s, latent_s = get_data(aois[0:1], model_name, crs=crs, norm=norm)
    data_2d_n, coord_n, latent_n = get_data(aois[-1:], model_name, crs=crs, norm=norm)
else:
    data_2d_s, coord_s, latent_s = get_data(aois[0:3], model_name, crs=crs, norm=norm)
    data_2d_n, coord_n, latent_n = get_data(aois[-3:], model_name, crs=crs, norm=norm)


In [4]:
latent_2d = np.full_like(np.zeros((7200,2800,5)), fill_value=float('NaN')) #7200, 2800

#latent_2d = np.full_like((7200,2800,5), float('NaN')) #7200, 2800
#recon_2d = np.zeros((7200,200*2,71)) #7200, 2800
#ml_2d = np.zeros((7200,200*2,71)) #7200, 2800
#ml_data = np.concatenate([np.load(f'data/{model_name}_ml_data_-10_0.npy'),np.load(f'data/{model_name}_ml_data_0_10.npy')])

for l in range(0,5):
    for i, (x,y) in enumerate(coord):
        latent_2d[x,y,l] = latent[i, l]
        #recon_2d[x,y,:] = recon[i, :]
        #ml_2d[x,y,:] = ml_data[i,:]

In [5]:
kmeans_name = f"/home/freya/Documents/Code/notebooks/map_data/kmeans/kmeans_{k}_{model_name}.pkl"
kmeans = cluster_with_kmeans(kmeans_name, data_2d, coord, latent, k)
cluster, cluster_2d = predict_kmeans(kmeans, data_2d, latent, coord)

# flag data
cluster_2d[np.mean(data_2d, axis=2) == 10] = 10
cluster_2d[np.mean(data_2d, axis=2) == -10] = -10

In [6]:
cluster_s, cluster_2d_s = predict_kmeans(kmeans, data_2d_s, latent_s, coord_s)
cluster_n, cluster_2d_n = predict_kmeans(kmeans, data_2d_n, latent_n, coord_n)

aoi_s = get_poles_aoi(area=area)
aoi_n = get_poles_aoi(area=area, region='north')

cluster_2d_s[np.mean(data_2d_s, axis=2) == 10] = 10
cluster_2d_s[np.mean(data_2d_s, axis=2) == -10] = -10
cluster_2d_n[np.mean(data_2d_n, axis=2) == 10] = 10
cluster_2d_n[np.mean(data_2d_n, axis=2) == -10] = -10

lat_edges, lat_centers = make_bins(aoi_s['lat_range'], int(
    30/aoi_s['d_lat']))  # 10 = 90-80, otherwise 30=90-60
long_edges, long_centers = make_bins(
    aoi_s['long_range'], int(360/aoi_s['d_long']))
loc_s = {'long_center': long_centers,
         'lat_center': lat_centers, }
df_2d_s = get_df_2d(np.array(cluster_2d_s), loc_s)
r_s, theta_s = get_polar_loc(df_2d_s, 'south')

lat_edges, lat_centers = make_bins(aoi_n['lat_range'], int(30/aoi_n['d_lat']))  # 10 = 90-80, otherwise 30=90-60
long_edges, long_centers = make_bins(
    aoi_n['long_range'], int(360/aoi_n['d_long']))
loc_n = {'long_center': long_centers,
         'lat_center': lat_centers, }
df_2d_n = get_df_2d(np.array(cluster_2d_n), loc_n)
r_n, theta_n = get_polar_loc(df_2d_n, 'north')

In [9]:
latent_s_2d = np.full_like(np.zeros((7200,600,5)), float('NaN')) #7200, 2800
for l in range(0,5):
    for i, (x,y) in enumerate(coord_s):
        latent_s_2d[x,y,l] = latent_s[i, l]
latent_n_2d = np.full_like(np.zeros((7200,600,5)), float('NaN')) #7200, 2800
for l in range(0,5):
    for i, (x,y) in enumerate(coord_n):
        latent_n_2d[x,y,l] = latent_n[i, l]

In [ ]:
s = sns.color_palette("viridis", n_colors=30)
cmap = matplotlib.colors.ListedColormap(s, "")
cmap
ymax = [6, 3, 5, 5, 4]
for l in range(0, 5):
    _min = -0.02
    _max = 0.6
    latent_2d[:, :, l][np.mean(data_2d, axis=2) == 10] = float('NaN')
    latent_2d[:, :, l][np.mean(data_2d, axis=2) == -10] = float('NaN')
    latent_s_2d[:, :, l][np.mean(data_2d_s, axis=2) == 10] = float('NaN')
    latent_s_2d[:, :, l][np.mean(data_2d_s, axis=2) == -10] = float('NaN')
    latent_n_2d[:, :, l][np.mean(data_2d_n, axis=2) == 10] = float('NaN')
    latent_n_2d[:, :, l][np.mean(data_2d_n, axis=2) == -10] = float('NaN')

    from math import cos, sin, atan2, sqrt
    fig = plt.figure(figsize=(15, 10))
    ax1 = fig.add_axes([0.0, 0.48, 0.8, 0.5])
    ax2 = fig.add_axes([0.0, 0.0, 0.45, 0.4], polar=True)
    ax3 = fig.add_axes([0.4, 0.0, 0.45, 0.4], polar=True)

    # left, bottom, width, height
    cbar_ax = fig.add_axes([0.85, 0.1, 0.015, 0.9])
    fig.subplots_adjust(right=0.8)

    sns.heatmap(np.rot90(latent_2d[:, :, l]), ax=ax1, cbar_ax=cbar_ax, cbar_kws={
                'label': 'Reflectance'}, vmin=np.nanmin(latent_2d[:, :, l]), vmax=ymax[l], cmap=cmap)
    # cbar_ax.set_yticks(np.linspace(0.5,k-0.5,k))
    # cbar_ax.set_yticklabels(np.linspace(1,k,k,dtype=int))

    ax1.set_xticks(np.linspace(0.5, np.rot90(
        latent_2d[:, :, l]).shape[1]-1.0, 19))
    ax1.set_xticklabels(
        f'{c:.1f}' for c in np.linspace(-180, 180, 19, dtype=int))
    ax1.set_yticks(np.linspace(0.5, np.rot90(
        latent_2d[:, :, l]).shape[0]-1.0, 15))
    ax1.set_yticklabels(f'{c:.1f}' for c in np.flip(
        np.linspace(-70, 70, 15, dtype=int)))
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')

    #loc = get_loc(aoi_s)

    df_2d_s = get_df_2d(np.array(latent_s_2d[:, :, l]), loc_s)
    r_s, theta_s = get_polar_loc(df_2d_s, 'south')

    dr = np.fabs((r_s[0] - r_s[1]) / 2)
    ax2.set_rlim(np.min(r_s) - dr, np.max(r_s) + dr)
    ax2.set_theta_direction(-1)
    ax2.set_theta_offset(np.pi / 2.0)
    ax2.pcolormesh(theta_s, r_s, df_2d_s, vmin=np.nanmin(
        latent_2d[:, :, l]), vmax=ymax[l], cmap=cmap)
    ax2.set_yticklabels([])

    #loc = get_loc(aoi_n)
    df_2d_n = get_df_2d(np.array(latent_n_2d[:, :, l]), loc_n)
    r_n, theta_n = get_polar_loc(df_2d_n, 'north')

    dr = np.fabs((r_n[0] - r_n[1]) / 2)
    ax3.set_rlim(np.min(r_n) - dr, np.max(r_n) + dr)
    ax3.set_theta_direction(1)
    ax3.set_theta_offset(np.pi / 2.0)
    ax3.set_theta_zero_location("S")
    ax3.set_xticklabels([Text(0.0, 0, '180°'),
                         Text(0.7853981633974483, 0, '135°'),
                         Text(1.5707963267948966, 0, '90°'),
                         Text(2.356194490192345, 0, '45°'),
                         Text(3.141592653589793, 0, '0°'),
                         Text(3.9269908169872414, 0, '315°'),
                         Text(4.71238898038469, 0, '270°'),
                         Text(5.497787143782138, 0, '225°')])
    print(ymax[l])
    ax3.pcolormesh(theta_n, r_n, df_2d_n, vmin=np.nanmin(
        latent_2d[:, :, l]), vmax=ymax[l], cmap=cmap)
    ax3.set_yticklabels([])
    plt.savefig(f'map_clusters_{k}_{area}_latent{l}.png',
                bbox_inches='tight', dpi=200)
    plt.show()

In [ ]:
# for l in range(0,5):
#     plt.figure(figsize=(15,5))
#     sns.heatmap(np.rot90(latent_2d[:,:,l]))
#     plt.gca().set_xticks(np.linspace(0.5, latent_2d[:,:,0].shape[1]-1.0, 19))
#     plt.gca().set_xticklabels(f'{c:.1f}' for c in np.linspace(-180, 180, 19,dtype=int))
#     plt.gca().set_yticks(np.linspace(0.5, latent_2d[:,:,0].shape[0]-1.0, 11))
#     plt.gca().set_yticklabels(f'{c:.1f}' for c in np.linspace(-75, 75, 11, dtype=int))
#     plt.gca().set_xlabel('Longitude')
#     plt.gca().set_ylabel('Latitude')
#     plt.savefig(f'{os.path.join(figure_dir, "latent")}/{model_name}_latent_{l}.png', dpi=200)
#     plt.show()